<a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a>

# 9.0 Enabling GPU within a Kubernetes (K8s) Cluster
## (part of Lab 3)

<img src="images/k8s/kubernetes_stack_0.png" style="float: right;">
In this notebook, you'll learn how to prepare a Kubernetes cluster for GPU acceleration full production deployment of conversational AI applications.<br><br>

**[9.1 Launch a K8s Cluster](#9.1-Launch-a-K8s-Cluster)<br>**
**[9.2 Deploy a CUDA Test Application](#9.2-Deploy-a-CUDA-Test-Application)<br>**
**[9.3 Add GPU Awareness to K8s](#9.3-Add-GPU-Awareness-to-K8s)<br>**
**[9.4 Interact with GPU Resources in K8s](#9.4-Interact-with-GPU-Resources-in-K8s)<br>**
&nbsp;&nbsp;&nbsp;&nbsp;[9.4.1 Exercise: Configure Pod](#9.4.1-Exercise:-Configure-Pod)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[9.4.2 Final Checks and Shutdown](#9.4.2-Final-Checks-and-Shutdown)<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[9.4.2.1 Exercise: Delete a Pod](#9.4.2.1-Exercise:-Delete-a-Pod)<br>

In the previous parts of the class, you deployed NVIDIA Riva using basic shell commands. As convenient as this method is during development, it becomes impractical when deploying to production, that is, when managing larger numbers of servers and services. 

[Kubernetes](https://kubernetes.io/), also known as K8s, is an open-source system for automating deployment, scaling, and management of containerized applications. 
In this part of the class, we will first launch a K8s cluster, enable the cluster for GPU acceleration and interact with those resources. This is our first step toward monitoring, managing, and deploying conversational AI applications in production. Monitoring and deployment will be covered in later notebooks.

### Notebook Dependencies
The steps in this notebook assume that you are starting with a clean environment.  Ensure that by stopping any previous Kubernetes installation and all docker containers, then looking at our environment's state. 

In [1]:
# Check running docker containers. This should be empty.
!docker ps

CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES


In [2]:
# If not empty,
# Clear Docker containers to start fresh...
!docker kill $(docker ps -q)

# Check for clean environment - this should be empty
!docker ps

"docker kill" requires at least 1 argument.
See 'docker kill --help'.

Usage:  docker kill [OPTIONS] CONTAINER [CONTAINER...]

Kill one or more running containers
CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES


In [3]:
# Deletes local Kubernetes cluster if it exists
!minikube delete

🙄  "minikube" profile does not exist, trying anyways.
💀  Removed all traces of the "minikube" cluster.


--- 
# 9.1 Launch a K8s Cluster

A [Kubernetes cluster](https://kubernetes.io/docs/concepts/overview/components/) consists of a set of worker machines (physical or virtual), called nodes, that run containerized applications. Every cluster has at least one worker node, though it can also support thousands of nodes! For this class, we will use [Minikube](https://minikube.sigs.k8s.io/docs/), which allows us to deploy a local and self-contained Kubernetes cluster with a single node. 

Review the class hardware resources available and launch the K8s cluster.

We can see details and status of the available GPU using the `nvidia-smi` command.

<img src="images/k8s/nvidia_smi.png">

In [4]:
# What GPU are we using and how much memory does it have?
!nvidia-smi

Thu Jul 27 07:38:57 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A10G         Off  | 00000000:00:1E.0 Off |                    0 |
|  0%   34C    P8    17W / 300W |      0MiB / 23028MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
# What type of CPU processor(s) are we using?
!cat /proc/cpuinfo | grep "model name"

model name	: AMD EPYC 7R32
model name	: AMD EPYC 7R32
model name	: AMD EPYC 7R32
model name	: AMD EPYC 7R32


In [6]:
# How many processors are available?
!nproc

4


In [7]:
# Launch the K8s cluster using Minikube
!minikube start --driver=none

😄  minikube v1.23.2 on Ubuntu 20.04 (docker/amd64)
✨  Using the none driver based on user configuration
👍  Starting control plane node minikube in cluster minikube
🤹  Running on localhost (CPUs=4, Memory=15818MB, Disk=297738MB) ...
ℹ️  OS release is Ubuntu 20.04.5 LTS
    > kubelet.sha256: 64 B / 64 B [--------------------------] 100.00% ? p/s 0s
    > kubeadm.sha256: 64 B / 64 B [--------------------------] 100.00% ? p/s 0s
    > kubectl.sha256: 64 B / 64 B [--------------------------] 100.00% ? p/s 0s
    > kubeadm: 43.71 MiB / 43.71 MiB [------------] 100.00% 95.66 MiB p/s 700msK/ K- 
    > kubectl: 44.73 MiB / 44.73 MiB [-----------] 100.00% 106.56 MiB p/s 600msK\ 
    > kubelet: 146.25 MiB / 146.25 MiB [-----------] 100.00% 98.14 MiB p/s 1.7sK/ 

    ▪ Generating certificates and keys ...
    ▪ Booting up control plane ...
    ▪ Configuring RBAC rules ...
🤹  Configuring local host environment ...

❗  The 'none' driver is designed for experts who need to integrate with an existing 

Once the cluster is successfully launched, we expect to see a number of containers running.  Check this by executing `docker ps` again.

In [8]:
# Listing the Kuberenetes components deployed
!docker ps

CONTAINER ID   IMAGE                  COMMAND                  CREATED          STATUS         PORTS     NAMES
6fc49fbf4a4c   e64579b7d886           "kube-apiserver --ad…"   9 seconds ago    Up 8 seconds             k8s_kube-apiserver_kube-apiserver-261913e3dab5_kube-system_165a0116aebb93e175f751846787d23c_0
7599860148c6   004811815584           "etcd --advertise-cl…"   9 seconds ago    Up 8 seconds             k8s_etcd_etcd-261913e3dab5_kube-system_28d51804f19478b4a9d2ff7e7e782917_0
9d38d77d1066   5425bcbd23c5           "kube-controller-man…"   9 seconds ago    Up 8 seconds             k8s_kube-controller-manager_kube-controller-manager-261913e3dab5_kube-system_d542bcea397b77607da4f7ef2f47861c_0
57008fb8c12d   b51ddc1014b0           "kube-scheduler --au…"   9 seconds ago    Up 8 seconds             k8s_kube-scheduler_kube-scheduler-261913e3dab5_kube-system_fdc4bb59feb740feccae609b8e626d94_0
da8c97f1e66a   k8s.gcr.io/pause:3.5   "/pause"                 19 seconds ago   Up 8 seconds   

We should now have access to the [kubectl command line tool](https://kubernetes.io/docs/reference/kubectl/overview/), which is used to interact with the cluster. List the nodes and services in the cluster using the `kubectl get` command:

In [9]:
# List nodes in the cluster
!kubectl get nodes

NAME           STATUS     ROLES                  AGE   VERSION
261913e3dab5   NotReady   control-plane,master   5s    v1.22.2


In [10]:
# List all services deployed
!kubectl get services

NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   4s


--- 
# 9.2 Deploy a CUDA Test Application

Next, we will deploy a simple GPU-accelerated application. This is a toy application which randomly generates two very large vectors and adds them. Print out the YAML configuration file needed to deploy the application:

In [11]:
# Set the configuration directory
CONFIG_DIR='/dli/task/kubernetes-config'

In [12]:
# Review the application we will deploy
!cat $CONFIG_DIR/gpu-pod.yaml

apiVersion: v1
kind: Pod
metadata:
  name: gpu-operator-test
spec:
  restartPolicy: OnFailure
  containers:
  - name: cuda-vector-add
    image: "nvidia/samples:vectoradd-cuda11.6.0"
    resources:
      limits:
         nvidia.com/gpu: 1

The main difference between a YAML file specifying a GPU-accelerated application compared to one specifying a non-GPU-accelerated application, is the configuration of the GPU resources required. In our case, we have created a basic configuration requesting a single NVIDIA GPU by setting `resources: limits: nvidia.com/gpu:` to 1. 

To deploy an application, execute the `kubectl apply` command, specifying the YAML configuration file with the `-f` file option.

In [13]:
# Deploy the application
!kubectl apply -f $CONFIG_DIR/gpu-pod.yaml

Error from server (Forbidden): error when creating "/dli/task/kubernetes-config/gpu-pod.yaml": pods "gpu-operator-test" is forbidden: error looking up service account default/default: serviceaccount "default" not found


Once deployed, we can observe the status of a pod created with `kubectl get`:

In [14]:
# Get the status of the pod deployed
!kubectl get pods gpu-operator-test

Error from server (NotFound): pods "gpu-operator-test" not found


At this stage, the application is in the "Pending" state. <br>
Why do you think this is case? Do you think its just the fact we have not given the application enough time to launch? Or do you think there are other reasons for this behavior? Try executing the same command again to see if the status changes.

In [15]:
# Checking again. Is it still pending?
!kubectl get pods gpu-operator-test

Error from server (NotFound): pods "gpu-operator-test" not found


So the application is indeed in the "Pending" state and it will remain like that irrespective of the amount of time we wait. Why? Begin to answer this by looking at the configuration of the available nodes (in our case we just have one). In particular, look for any NVIDIA-specific configuration using the `kubectl describe` command, as this will help us identify GPU resources:

In [16]:
# Can we see the GPU?
!kubectl describe nodes

Name:               261913e3dab5
Roles:              control-plane,master
Labels:             beta.kubernetes.io/arch=amd64
                    beta.kubernetes.io/os=linux
                    kubernetes.io/arch=amd64
                    kubernetes.io/hostname=261913e3dab5
                    kubernetes.io/os=linux
                    minikube.k8s.io/commit=0a0ad764652082477c00d51d2475284b5d39ceed
                    minikube.k8s.io/name=minikube
                    minikube.k8s.io/updated_at=2023_07_27T07_39_36_0700
                    minikube.k8s.io/version=v1.23.2
                    node-role.kubernetes.io/control-plane=
                    node-role.kubernetes.io/master=
                    node.kubernetes.io/exclude-from-external-load-balancers=
Annotations:        kubeadm.alpha.kubernetes.io/cri-socket: /var/run/dockershim.sock
                    volumes.kubernetes.io/controller-managed-attach-detach: true
CreationTimestamp:  Thu, 27 Jul 2023 07:39:33 +0000
Taints:             

Can you find anything? Try again, filtering the output with `grep`:

In [17]:
# Let's look for the lines containing the word "nvidia"
!kubectl describe nodes | grep nvidia

We did not find anything. That would explain why the application is still pending. Our cluster is not aware of the presence of the GPU.  The cluster is unable to schedule the execution since our YAML required GPU resources, but they are for all intents and purposes unavailable. We need to add the NVIDIA GPU device plugin.

--- 
# 9.3 Add GPU Awareness to K8s
To take advantage of GPU acceleration on Kubernetes, install the [NVIDIA GPU plugin](https://kubernetes.io/docs/tasks/manage-gpus/scheduling-gpus/#deploying-nvidia-gpu-device-plugin) to the cluster. Before adding it, look at the status without the plugin  with `kubectl get`:

In [18]:
# Try to find the GPU device plugin. Not there 
!kubectl get pods -A

NAMESPACE     NAME                                   READY   STATUS    RESTARTS   AGE
kube-system   etcd-261913e3dab5                      0/1     Running   0          5s
kube-system   kube-apiserver-261913e3dab5            0/1     Running   0          5s
kube-system   kube-controller-manager-261913e3dab5   0/1     Pending   0          5s
kube-system   kube-scheduler-261913e3dab5            0/1     Running   0          6s
kube-system   storage-provisioner                    0/1     Pending   0          4s


To install the NVIDIA GPU plugin, we can use the Kubernetes package manager [Helm](https://helm.sh/).

In [19]:
# Install the device plugin with the Helm package manager
!helm repo add nvdp https://nvidia.github.io/k8s-device-plugin \
   && helm repo update
!helm upgrade -i nvdp nvdp/nvidia-device-plugin \
  --namespace nvidia-device-plugin \
  --create-namespace \
  --version 0.13.0

"nvdp" has been added to your repositories
Hang tight while we grab the latest from your chart repositories...
...Successfully got an update from the "nvdp" chart repository
Update Complete. ⎈Happy Helming!⎈
Release "nvdp" does not exist. Installing it now.
NAME: nvdp
LAST DEPLOYED: Thu Jul 27 07:39:42 2023
NAMESPACE: nvidia-device-plugin
STATUS: deployed
REVISION: 1
TEST SUITE: None


Check the status again to make sure the plugin was deployed:

In [20]:
# Now the device plugin "nvidia-device-plugin-*" should be "Running" after a "ContainerCreating" status
!kubectl get pods -A

NAMESPACE     NAME                                   READY   STATUS    RESTARTS   AGE
kube-system   etcd-261913e3dab5                      1/1     Running   0          6s
kube-system   kube-apiserver-261913e3dab5            0/1     Running   0          6s
kube-system   kube-controller-manager-261913e3dab5   0/1     Pending   0          6s
kube-system   kube-scheduler-261913e3dab5            1/1     Running   0          7s
kube-system   storage-provisioner                    0/1     Pending   0          5s


We should now see the NVIDIA-specific configuration listed against the nodes:

In [21]:
# Now we should see Allocable GPUs
!kubectl describe nodes

Name:               261913e3dab5
Roles:              control-plane,master
Labels:             beta.kubernetes.io/arch=amd64
                    beta.kubernetes.io/os=linux
                    kubernetes.io/arch=amd64
                    kubernetes.io/hostname=261913e3dab5
                    kubernetes.io/os=linux
                    minikube.k8s.io/commit=0a0ad764652082477c00d51d2475284b5d39ceed
                    minikube.k8s.io/name=minikube
                    minikube.k8s.io/updated_at=2023_07_27T07_39_36_0700
                    minikube.k8s.io/version=v1.23.2
                    node-role.kubernetes.io/control-plane=
                    node-role.kubernetes.io/master=
                    node.kubernetes.io/exclude-from-external-load-balancers=
Annotations:        kubeadm.alpha.kubernetes.io/cri-socket: /var/run/dockershim.sock
                    volumes.kubernetes.io/controller-managed-attach-detach: true
CreationTimestamp:  Thu, 27 Jul 2023 07:39:33 +0000
Taints:             

In [22]:
# Let's look for the lines containing the word nvidia
!kubectl describe nodes | grep nvidia

As we deployed the GPU device plugin, what do you think happened to our application?

In [23]:
# Let's check the application again
!kubectl get pods gpu-operator-test

Error from server (NotFound): pods "gpu-operator-test" not found


Our application executed successfully when the GPU resources became available. In fact, it has now completed so we can have a look at its execution logs with `kubectl logs`:

In [24]:
# Let's look at the output
!kubectl logs gpu-operator-test

Error from server (NotFound): pods "gpu-operator-test" not found


Check the list of Helm charts installed with the `helm list` command (see the [Helm documentation](https://helm.sh/docs/helm/helm_list/)). The `--filter` option allows filtering by name.  Use the `--output` option to specify the output format ("json", "table", or "yaml").  

Now, let's delete the Kubernetes pod `gpu-operator-test`:

In [25]:
# Let's delete the pod
!kubectl delete pod gpu-operator-test 

Error from server (NotFound): pods "gpu-operator-test" not found


Congratulations! You deployed a GPU accelerated applicaiton with Kuberenetes. So far, we have specified that we want a single GPU without specifying which GPU we want.

--- 
# 9.4 Interact with GPU Resources in K8s

Now, let's see how to get more control over the GPU-accelerated cluster. Being able to control the GPU type, or the MIG ([Multi-Instance GPU](https://www.nvidia.com/en-us/technologies/multi-instance-gpu/)) partition on an Ampere GPU is very important as GPUs vary in terms of computational capability, memory, and cost. The MIG allows users to fragment the GPU into as many as 7 (on A100) partitions. This allows more granular control over the resources in the cluster and better application isolation. 

In order to control the GPU type, we'll add the `gpu-feature-discovery` plugin and deploy it with Helm. This plugin can be configured with several options, as described in the [gpu-feature-discovery repository](https://github.com/NVIDIA/gpu-feature-discovery#deployment-via-helm). One of the most interesting options when working with Ampere GPUs is the ability to support MIG partitions. The feature discovery plugin can be deployed with the following configurable features:


|Feature|Description|Default|
|-|-|-|
|`failOnInitError`|Fail if there is an error during initialization of any label sources|"true"|
|`sleepInterval`|Time to sleep between labeling|"60s"|
|`migStrategy`|Pass the desired strategy for labeling MIG devices on GPUs that support it [none | single | mixed]|"none"|
|`nfd.deploy`|When set to true, deploy NFD as a subchart with all of the proper parameters set for it|"true"|

In this class, we are not using Ampere GPUs, so we will do a simple install:

In [26]:
# Install feature discovery with the Helm package manager
!helm repo add nvgfd https://nvidia.github.io/gpu-feature-discovery \
    && helm repo update
!helm upgrade -i nvgfd nvgfd/gpu-feature-discovery \
  --version 0.7.0 \
  --namespace gpu-feature-discovery \
  --create-namespace

"nvgfd" has been added to your repositories
Hang tight while we grab the latest from your chart repositories...
...Successfully got an update from the "nvdp" chart repository
...Successfully got an update from the "nvgfd" chart repository
Update Complete. ⎈Happy Helming!⎈
Release "nvgfd" does not exist. Installing it now.
NAME: nvgfd
LAST DEPLOYED: Thu Jul 27 07:39:44 2023
NAMESPACE: gpu-feature-discovery
STATUS: deployed
REVISION: 1
TEST SUITE: None


Let's look at additional information that we have about our system:

In [27]:
# Looking for all of the NVIDIA related information
!kubectl describe nodes | grep "nvidia.com" -A 15

You should see a wide range of GPU-specific information, including the driver and CUDA information, as well as which GPU is in use from `nvidia.com/gpu.product`.

This is probably an NVIDIA A10, unless you are running the class on an alternative GPU. Recall that we deployed our test application `gpu-operator-test` with a generic "GPU".  It is possible to deploy it with more specific information regarding the GPU. 

A new YAML file, `gpu-pod-A10.yaml`, is already prepared. Let's inspect it first:

In [28]:
# Review the application we are deploying
!cat $CONFIG_DIR/gpu-pod-A10.yaml

apiVersion: v1
kind: Pod
metadata:
  name: gpu-operator-test-a100
spec:
  restartPolicy: OnFailure
  containers:
  - name: cuda-vector-add
    image: "nvidia/samples:vectoradd-cuda11.6.0"
    resources:
      limits:
         nvidia.com/gpu: 1
  nodeSelector: 
    nvidia.com/gpu.product: A100-SXM4-40GB 

As you might have noticed, the YAML was configured to deploy on an A100 GPU, which is not available in the class. Go ahead and deploy the application anyway.

In [29]:
!kubectl apply -f $CONFIG_DIR/gpu-pod-A10.yaml

Error from server (Forbidden): error when creating "/dli/task/kubernetes-config/gpu-pod-A10.yaml": pods "gpu-operator-test-a100" is forbidden: error looking up service account default/default: serviceaccount "default" not found


In [30]:
!kubectl get pods gpu-operator-test-a100

Error from server (NotFound): pods "gpu-operator-test-a100" not found


Just as we saw in the earlier non-GPU case, the deployment is in the "Pending" state and it will remain in this state until an A100 GPU becomes available or it is terminated. 

## 9.4.1 Exercise: Configure Pod

Modify the YAML file and deploy the `gpu-operator-test` application on the correct GPU.
Open the [gpu-pod-A10.yaml](kubernetes-config/gpu-pod-A10.yaml) config file and make those changes:
* Change the pod name to "gpu-operator-test-a10"
* Set the GPU product to the GPU name you found earlier (such as "NVIDIA-A10G") instead of the A100

Check your work against the [solution](solutions/ex9.4.1.yaml) before moving on:

In [31]:
# TODO modify gpu-pod-A10.yaml so that this cell verifies changes are correct
# Check your work - you'll get no output if the files match
!diff $CONFIG_DIR/gpu-pod-A10.yaml solutions/ex9.4.1.yaml

4c4
<   name: gpu-operator-test-a100
---
>   name: gpu-operator-test-a10
14c14
<     nvidia.com/gpu.product: A100-SXM4-40GB 
\ No newline at end of file
---
>     nvidia.com/gpu.product: NVIDIA-A10G 
\ No newline at end of file


In [32]:
# Quick Fix!
!cp solutions/ex9.4.1.yaml $CONFIG_DIR/gpu-pod-A10.yaml

Next, deploy the `gpu-operator-test-a10` pod using the modified [gpu-pod-A10.yaml](kubernetes-config/gpu-pod-A10.yaml).

In [33]:
!kubectl apply -f $CONFIG_DIR/gpu-pod-A10.yaml

Error from server (Forbidden): error when creating "/dli/task/kubernetes-config/gpu-pod-A10.yaml": pods "gpu-operator-test-a10" is forbidden: error looking up service account default/default: serviceaccount "default" not found


## 9.4.2 Final Checks and Shutdown
It might take several seconds, but the application should deploy and finish successfully.  Rerun the next cell until the status shows the test is "completed".

In [34]:
# Get the status of the pod deployed
!kubectl get pods gpu-operator-test-a10

Error from server (NotFound): pods "gpu-operator-test-a10" not found


In [35]:
# Let's look at the output
!kubectl logs gpu-operator-test-a10

Error from server (NotFound): pods "gpu-operator-test-a10" not found


### 9.4.2.1 Exercise: Delete a Pod

Delete the Kubernetes pod `gpu-operator-test-a10`. Check the [solution](solutions/ex9.4.2.ipynb) before moving on:

In [36]:
# TODO delete the pod
!kubectl #FIXME

kubectl controls the Kubernetes cluster manager.

 Find more information at:
https://kubernetes.io/docs/reference/kubectl/overview/

Basic Commands (Beginner):
  create        Create a resource from a file or from stdin
  expose        Take a replication controller, service, deployment or pod and
expose it as a new Kubernetes service
  run           Run a particular image on the cluster
  set           Set specific features on objects

Basic Commands (Intermediate):
  explain       Get documentation for a resource
  get           Display one or many resources
  edit          Edit a resource on the server
  delete        Delete resources by file names, stdin, resources and names, or
by resources and label selector

Deploy Commands:
  rollout       Manage the rollout of a resource
  scale         Set a new size for a deployment, replica set, or replication
controller
  autoscale     Auto-scale a deployment, replica set, stateful set, or
replication controller

Cluster Management Commands

Before moving forward to the next notebook, shut down K8s and clean up the docker environment.

In [37]:
# Shut down K8s
!minikube delete
# Shut down running docker containers
!docker kill $(docker ps -q)
# Check for clean environment - this should be empty
!docker ps

🔄  Uninstalling Kubernetes v1.22.2 using kubeadm ...
🔥  Deleting "minikube" in none ...
💀  Removed all traces of the "minikube" cluster.
"docker kill" requires at least 1 argument.
See 'docker kill --help'.

Usage:  docker kill [OPTIONS] CONTAINER [CONTAINER...]

Kill one or more running containers
CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES


---
<h2 style="color:green;">Congratulations!</h2>

In this notebook, you have:
- Launched a K8s cluster
- Interacted with K8s using `kubectl`
- Installed plugins with Helm
- Enabled GPU acceleration and GPU feature discovery
- Deployed an application

Next, you'll monitor activity on the cluster. Move on to [Monitoring GPU within Kubernetes Cluster](010_K8s_Monitor.ipynb).

<a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a>